<a href="https://colab.research.google.com/github/chimaOkwuokei/ISEND-PAU/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import random
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import (
    Activation,
    AveragePooling2D,
    BatchNormalization,
    Conv2D,
    MaxPooling2D,
    ZeroPadding2D,
)
from tensorflow.keras.models import Model
from utils import stack1

# Set seed for reproducibility
def set_seed(seed_value=42):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

set_seed()  # Set seed for consistent results


# Fully convolutional ResNet50 model
def fully_convolutional_resnet50(input_shape, num_classes=1000, pretrained_resnet=True, use_bias=True):
    # Initialize input layer
    img_input = Input(shape=input_shape)

    # Define basic ResNet50 model
    x = ZeroPadding2D(padding=((3, 3), (3, 3)), name="conv1_pad")(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=use_bias, name="conv1_conv")(x)
    x = BatchNormalization(axis=3, epsilon=1.001e-5, name="conv1_bn")(x)
    x = Activation("relu", name="conv1_relu")(x)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name="pool1_pad")(x)
    x = MaxPooling2D(3, strides=2, name="pool1_pool")(x)

    # Stack the ResNet50 residual blocks
    x = stack1(x, 64, 3, stride1=1, name="conv2")
    x = stack1(x, 128, 4, name="conv3")
    x = stack1(x, 256, 6, name="conv4")
    x = stack1(x, 512, 3, name="conv5")

    # Add average pooling layer
    x = AveragePooling2D(pool_size=7)(x)

    # Final convolutional layer
    conv_layer_final = Conv2D(filters=num_classes, kernel_size=1, use_bias=use_bias, name="last_conv")(x)

    # Build model
    model = Model(inputs=img_input, outputs=[conv_layer_final])

    # Load pretrained weights if necessary
    if pretrained_resnet:
        resnet50_extractor = ResNet50(include_top=False, weights="imagenet")
        # Transfer weights from the pretrained ResNet50 model
        for layer in resnet50_extractor.layers:
            if 'conv' in layer.name or 'bn' in layer.name:
                model.get_layer(layer.name).set_weights(layer.get_weights())

    return model


if __name__ == "__main__":

    # Read ImageNet class ids to a list of labels
    with open("imagenet_classes.txt") as f:
        labels = [line.strip() for line in f.readlines()]

    # Read and preprocess the input image
    original_image = cv2.imread("camel.jpg")
    image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # Ensure the image is compatible with the model
    if image.shape[0] % 32 != 0 or image.shape[1] % 32 != 0:
        print("Warning: Image size not divisible by 32, may affect performance.")

    # Preprocess input as required by ResNet50
    image = preprocess_input(image)

    # Expand image to include batch dimension
    image = tf.expand_dims(image, 0)

    # Set seed before predictions
    set_seed()

    # Load fully convolutional ResNet50 model with pre-trained ImageNet weights
    model = fully_convolutional_resnet50(input_shape=(image.shape[-3:]), num_classes=1000)

    preds = model.predict(image)
    preds = tf.transpose(preds, perm=[0, 3, 1, 2])
    preds = tf.nn.softmax(preds, axis=1)
    print("Response map shape : ", preds.shape)

    # Find the class with the maximum score in the n × m output map
    pred = tf.math.reduce_max(preds, axis=1)
    class_idx = tf.math.argmax(preds, axis=1)
    print(class_idx)

    row_max = tf.math.reduce_max(pred, axis=1)
    row_idx = tf.math.argmax(pred, axis=1)

    col_idx = tf.math.argmax(row_max, axis=1)

    predicted_class = tf.gather_nd(
        class_idx, (0, tf.gather_nd(row_idx, (0, col_idx[0])), col_idx[0]),
    )

    # Print top predicted class
    print("Predicted Class : ", labels[predicted_class], predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Response map shape :  (1, 1000, 3, 8)
tf.Tensor(
[[[555 602 263 602 146 555 146 555]
  [820 444 444 123 444 444  25 429]
  [517 239 239 506 506 583 290 785]]], shape=(1, 3, 8), dtype=int64)
Predicted Class :  coil, spiral, volute, whorl, helix tf.Tensor(506, shape=(), dtype=int64)
